In [1]:
import numpy as np
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

In [2]:
def calculate_uncertainty_LeastConfident(X, model):
    # Get predicted probabilities for each class
    predicted_probs = model.predict_proba(X)    
    # Calculate uncertainty scores as the least confident probability
    least_confident = 1 - np.max(predicted_probs, axis=1)    
    return least_confident

def calculate_uncertainty_SmallestMargin(X, model):
    # Get predicted probabilities for each class
    predicted_probs = model.predict_proba(X)    
    # Sort the predicted probabilities in descending order
    sorted_probs = np.sort(predicted_probs, axis=1)[:, ::-1]    
    # Calculate uncertainty scores as the difference between the top two probabilities
    smallest_margin = sorted_probs[:, 0] - sorted_probs[:, 1]    
    return smallest_margin

def calculate_uncertainty_Entropy(X, model):
    # Get predicted probabilities for each class
    predicted_probs = model.predict_proba(X)    
    # Check for zero probabilities and replace them with a small value
    predicted_probs[predicted_probs == 0] = 1e-10
    # Calculate uncertainty scores using the entropy
    entropy = -np.sum(predicted_probs * np.log2(predicted_probs), axis=1)
    # Handle the case of uniform probabilities
    entropy[np.isnan(entropy)] = 0    
    return entropy

def select_uncertain_samples_LC(X, model, n_samples=1):
    uncertainty_scores=calculate_uncertainty_LeastConfident(X, model)   
    # Select the indices of the most uncertain samples
    uncertain_indices = np.argsort(uncertainty_scores)[-n_samples:]    
    return uncertain_indices,uncertainty_scores

def select_uncertain_samples_SM(X, model, n_samples=1):
    uncertainty_scores=calculate_uncertainty_SmallestMargin(X, model)   
    # Select the indices of the most uncertain samples
    #uncertain_indices = np.argsort(uncertainty_scores)[:n_samples]    
    uncertain_indices = np.argsort(uncertainty_scores)[::-1][-n_samples:]
    #np.where(uncertainty_scores == np.min(uncertainty_scores))    
    return uncertain_indices ,uncertainty_scores

def select_uncertain_samples_Entropy(X, model, n_samples=1):
    uncertainty_scores=calculate_uncertainty_Entropy(X, model_Entropy)   
    # Select the indices of the most uncertain samples
    uncertain_indices = np.argsort(uncertainty_scores)[-n_samples:]    
    np.where(uncertainty_scores == np.max(uncertainty_scores))    
    return uncertain_indices,uncertainty_scores

In [3]:
# Load the Iris dataset
iris = load_iris()
X, y = iris.data[:, :2], iris.target
X, y = shuffle(X, y, random_state=42)

In [ ]:
model_Entropy = LogisticRegression()
model_LC = LogisticRegression()
model_SM = LogisticRegression()
# Select initial labeled samples randomly
labeled_indices = np.random.choice(X.shape[0], size=5, replace=False)
unlabeled_indices = np.setdiff1d(np.arange(X.shape[0]), labeled_indices)

labeled_indices_Entropy=labeled_indices
labeled_indices_LC=labeled_indices
labeled_indices_SM=labeled_indices

unlabeled_indices_Entropy=unlabeled_indices
unlabeled_indices_LC=unlabeled_indices
unlabeled_indices_SM=unlabeled_indices
# Start the active learning loop
n_iterations = 5
n_samples_per_iteration = 1